### Qubit-Only Cost

The beamsplitter operator acting on two modes is defined as $BS(\theta, \phi)_{ij} = e^{-i\theta\left(e^{i\phi}a^{\dagger}_i a_j + e^{-i\phi}a_i a^{\dagger}_j\right)}$, with the generating Hamiltonian $H = \theta\left(e^{i\phi}a^{\dagger}_i a_j + e^{-i\phi}a_i a^{\dagger}_j\right)$. We will analyze the cost of implementing this for $n < 50$ logical qubits.

Using the Fock Binary encoding, where the Fock state $|n\rangle$ maps to the qubit state $n$ in base-2, we calculate the cost of a beamsplitter between two Bosonic states $|1\rangle$ ($|0\dots01\rangle$ in this encoding) by $\theta = \phi = 1$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit_algorithms import TimeEvolutionProblem
from qiskit_algorithms import TrotterQRTE
from qiskit.quantum_info import Statevector
from qiskit import transpile
from qiskit.providers.fake_provider import GenericBackendV2
from scipy.sparse import csr_matrix, kron

import sys
from pathlib import Path

# Add the `playground` directory to the Python path
playground_path = Path.cwd().parent.parent  # Move two levels up to `playground`
sys.path.append(str(playground_path))
# Homemade functions
from bosonic_operator_costs.all_qubit_utils import create_annihilation_operator, create_creation_operator, simulate_system

In [ ]:
def beamsplitter_hamiltonian(n_max, theta=1, phi=1):
    """
    Construct the Hamiltonian H = θ(e^{iφ}a^†b + e^{-iφ}ab^†).

    Parameters:
        n_max (int): The Fock cutoff.
        theta (float): The θ parameter.
        phi (float): The φ parameter.

    Returns:
        np.ndarray: The Hamiltonian matrix as a NumPy array.
    """
    # Operators for mode 'a'
    a = create_annihilation_operator(n_max)
    a_dag = create_creation_operator(n_max)

    # Operators for mode 'b'
    b = create_annihilation_operator(n_max)
    b_dag = create_creation_operator(n_max)

    # Tensor products to place operators in the correct subspaces
    a_dag_b = np.kron(a_dag, b)  # a^† ⊗ b
    a_b_dag = np.kron(a, b_dag)  # a ⊗ b^†

    # Construct the Hamiltonian
    H = theta * (np.exp(1j * phi) * a_dag_b + np.exp(-1j * phi) * a_b_dag)

    # Convert the result to a NumPy array and return
    return np.array(H)

def define_initial_state(num_modes, n_max):
    """
    Define the initial state vector for the system.

    Parameters:
        n_qubits (int): The number of qubits in the system.

    Returns:
        Statevector: The initial quantum state.
    """
    n_qubits = num_modes * int(np.ceil(np.log2(n_max + 1)))
    initial_state_vector = np.zeros(2**n_qubits, dtype=complex)
    initial_state_vector[2] = 1.0

    return Statevector(initial_state_vector)

In [ ]:
cutoffs = [2**n - 1 for n in range(1, 6)]
num_modes = 2  

# Initialize lists to store results
rz_gates = []
cnot_gates = []
total_gates = []
total_depths = []
rotation_gate_depths = []
cnot_gate_depths = []

# Loop through each cutoff and simulate the system
for n_max in cutoffs:
    #Hamiltonian and state for current cutoff
    hamiltonian = beamsplitter_hamiltonian(n_max) 
    state = define_initial_state(num_modes, n_max)

    # Simulate the system and collect results
    results = simulate_system(
        hamiltonian=hamiltonian,
        initial_state=state,
        num_modes=num_modes,
        n_max=n_max
    )
    # Collect gate counts and depths
    rz_gates.append(results["gate_counts"].get('rz', 0))
    cnot_gates.append(results["gate_counts"].get('cx', 0))
    total_gates.append(sum(results["gate_counts"].values()))
    total_depths.append(results["total_depth"])
    rotation_gate_depths.append(results["rz_depth"])
    cnot_gate_depths.append(results["cnot_depth"])

In [ ]:
from scipy.optimize import curve_fit

# Plot labels and data
plots = [
    {"y": cnot_gates, "label": "CNOT Gates", "color": "orange", "ylabel": "CNOT Gates"},
    {"y": rz_gates, "label": "Rotation Gates", "color": None, "ylabel": "Rotation Gates"},
    {"y": cnot_gate_depths, "label": "CNOT Depth", "color": "green", "ylabel": "CNOT Depth"},
    {"y": rotation_gate_depths, "label": "Rotation Gate Depth", "color": "red", "ylabel": "Rotation Gate Depth"},
]
# Define power law function for fitting
def power_law(x, k, n):
    return k * x**n

# Create subplots
plt.figure(figsize=(12, 8))

for i, plot in enumerate(plots, 1):
    y_data = plot["y"]
    
    # Perform curve fitting
    params, _ = curve_fit(power_law, cutoffs, y_data)
    k, n = params
    
    # Print scaling parameters for each dataset
    print(f"{plot['label']} Scaling: k={k:.2f}, n={n:.2f}")
    
    # Plot original data and fit
    plt.subplot(2, 2, i)
    plt.plot(cutoffs, y_data, 'o-', label=f"{plot['label']} Data", color=plot["color"])
    plt.plot(cutoffs, power_law(cutoffs, *params), '--', 
         label=f"Fit: $y = {k:.2f}x^{{{n:.2f}}}$", color=plot["color"]) 
    plt.yscale('log')  # Set log scale

    plt.xlabel("Cutoff")
    plt.ylabel(plot["ylabel"])
    plt.grid()
    # plt.legend()

# Adjust layout and show the plots
plt.tight_layout()
plt.show()